---
<div class="alert alert-block alert-success">

## Комментарий от наставника

Спасибо за доработки, сейчас все в порядке. Проект засчитан, поздравляю!
</div>

---

---

<div class="alert alert-block alert-danger">
    

## Комментарий от наставника

Привет! Спасибо за проект. Сейчас остались некоторые вопросы и замечания, выделены в коде красными блоками. Исправь или дополнительно прокомментируй их, пожалуйста.
</div>
    
---

## Описание задачи
Сегментировать покупателей по профилю потребления

- Проведите исследовательский анализ данных
- Сегментируйте покупателей на основе истории их покупок
- Сформулируйте и проверьте статистические гипотезы

### Описание данных

Датасет описывает транзакции интернет-магазина товаров для дома и быта «Пока все ещё тут».

Колонки в  *ecommerce_dataset.csv* :

- `date` — дата заказа;
- `customer_id` — идентификатор покупателя;
- `order_id` — идентификатор заказа;
- `product` — наименование товара;
- `quantity` — количество товара в заказе;
- `price` — цена товара.

## Декомпозиция задачи. План действий

1. Просмотр датасета и предобработка(приведение к правильному типу данных, работа с дубликатами и пропусками)
2. Исследовательский анализ(EDA)
    * Посмотреть на то, как часто покупают
    * Возвращаемость когорт
    * Сколько покупают в сутки/месяц
    * Сколько в среднем получается покупок? Много ли заказывают или докупают что-то?
    * Разброс по сумме заказа
* Сегментация покупателей на основе покупок 
    * Как часто покупают
    * На какие суммы покупают
* Формулировка гипотез и их проверка
    * Гипотеза о равенстве среднего значения суммы корзины разных сегментов
* Презентация и дашборд

## Материалы
* [Презентация](https://drive.google.com/file/d/1tjHw9n5SeVBzPuwAIJJ3tve8cn81BF1C/view?usp=sharing)

<div class="alert alert-block alert-warning">
    
Оглавление:
    
1. [Предобработка данных](#step1)
2. [Исследовательский анализ данных EDA](#step2)
3. [Сегментация покупателей на сонове покупок](#step3)
4. [Формулировка гипотез и их проверка](#step4)
5. [Общий вывод к проекту и рекомендации](#step5)
    
</div> 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math as mth
from scipy import stats as st

## Предобработка данных <a id='step1'></a>

In [3]:
#Локалка
df = pd.read_csv('C://Users/Ритер/Desktop/Практикум/Диплом/Ecomm/ecommerce_dataset.csv')

#Практикум
#df = pd.read_csv('/datasets/ecommerce_dataset.csv')

FileNotFoundError: [Errno 2] File b'C://Users/\xd0\xa0\xd0\xb8\xd1\x82\xd0\xb5\xd1\x80/Desktop/\xd0\x9f\xd1\x80\xd0\xb0\xd0\xba\xd1\x82\xd0\xb8\xd0\xba\xd1\x83\xd0\xbc/\xd0\x94\xd0\xb8\xd0\xbf\xd0\xbb\xd0\xbe\xd0\xbc/Ecomm/ecommerce_dataset.csv' does not exist: b'C://Users/\xd0\xa0\xd0\xb8\xd1\x82\xd0\xb5\xd1\x80/Desktop/\xd0\x9f\xd1\x80\xd0\xb0\xd0\xba\xd1\x82\xd0\xb8\xd0\xba\xd1\x83\xd0\xbc/\xd0\x94\xd0\xb8\xd0\xbf\xd0\xbb\xd0\xbe\xd0\xbc/Ecomm/ecommerce_dataset.csv'

In [ ]:
def vision_df(list_df):
    cnt = 1
    for i in list_df:
        print('----------------------------DataFrame ', cnt, '-------------------------------')
        print()
        print('Info', i.info())
        print()
        print()
        for col in i.columns:
            print(f'Duplicate in {col}: {i[col].duplicated().sum()}')
        print()
        display(i.head())
        print('------------------------------------------------------------------------')
        print('------------------------------------------------------------------------')
        cnt += 1

In [ ]:
vision_df([df])

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d%H')

In [ ]:
df.info()

Пропусков в датасете нет. Из проблем с типом данных есть только колонка с датой. Дубликаты обоснованны, т.к. у нас формат такой, что каждая строка это отдельный товар со своими характеристиками(ценой, датой заказа, id заказа и покупателя). По названиям колонок тоже все ок.

## Исследовательский анализ данных EDA <a id='step2'></a>
* Сколько покупают в сутки/по дням недели/месяц
* Возвращаемость когорт
* Сколько в среднем получается покупок? Много ли заказывают или докупают что-то?
* Разброс по сумме заказа

## Когда покупают
Сначала посмотрим какой временной промежуток у нас есть. Выделим дату, чтобы можно было посмотреть как распределены покупки по дням. Тоже самое можно сделать с часами.

In [ ]:
df['time'] = df['date'].dt.hour
df['month'] = df['date'].astype('datetime64[M]')
df['weekday'] = df['date'].dt.weekday
df['month_number'] = df['month'].dt.month
df['date'] = df['date'].dt.date

In [ ]:
df.head()

In [ ]:
print('Временной период с {} по {}'.format(df['date'].min(), df['date'].max()))

Получается что у нас данные за год с начала октября 2018 по конец октября 2019 г.

In [ ]:
df.groupby('month').agg({'order_id':'nunique'}).reset_index()

In [ ]:
fig = plt.subplots(figsize=(15,8))

x = df.groupby('month').agg({'order_id':'nunique'}).reset_index()['month']
y = df.groupby('month').agg({'order_id':'nunique'}).reset_index()['order_id']

plt.plot(x, y)
plt.grid(True)
plt.title('Количество заказов помесяцно за год')
plt.xlabel('Месяц')
plt.ylabel('Заказы, шт.')
plt.show()

In [ ]:
fig = plt.subplots(figsize=(15,8))

x = ['понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота', 'воскресенье']
y = df.groupby('weekday').agg({'order_id':'nunique'}).reset_index()['order_id']

plt.plot(x, y)
plt.grid(True)
plt.title('Количество заказов по дням недели')
plt.xlabel('День недели')
plt.ylabel('Заказы, шт.')
plt.show()

In [ ]:
fig = plt.subplots(figsize=(15,8))

x = df.groupby('time').agg({'order_id':'nunique'}).reset_index()['time']
y = df.groupby('time').agg({'order_id':'nunique'}).reset_index()['order_id']

plt.plot(x, y)
plt.grid(True)
plt.title('Количество заказов по часам')
plt.xlabel('Час')
plt.ylabel('Заказы, шт.')
plt.xticks(np.arange(0, 24, step=1))
plt.show()

In [ ]:
print('Среднее количество покупок в месяц: {:.0f}'.format(df.groupby('month').agg({'order_id':'nunique'})['order_id'].mean()))
print('Медиана количества покупок в месяц: {:.0f}'.format(df.groupby('month').agg({'order_id':'nunique'})['order_id'].median()))
print('-----------------------------------------')
print('Среднее количество покупок в день: {:.0f}'.format(df.groupby('date').agg({'order_id':'nunique'})['order_id'].mean()))
print('Медиана количества покупок в день: {:.0f}'.format(df.groupby('date').agg({'order_id':'nunique'})['order_id'].median()))

### Вывод по тому когда покупают
* Пиковые месяцы для нашего интернет-магазина приходятся на декабрь и февраль. Есть также всплеск активности в апреле. "Мертвые месяцы" - январь и июнь
* По дням недели активность, начиная с понедельника, падает до субботы. Видимо, в будние дни проще заказать
* Наибольший наплыв заказов приходиться на 11 часов

Может утром в понедельник не хочется работать и лучше сделать заказ в интернет-магазине?

Среднее и медианное значения количества заказов говорят о том, что в день обычно приходит 8 заказов. Среднее и медиана количества заказов в месяц немного разнятся(217 и 201 шт. соответственно) из-за того, что есть "активные" месяцы продажи.

---
<div class="alert alert-block alert-success">

## Комментарий от наставника

В этом разделе все верно.
</div>

---

## Заказы

In [ ]:
print('Среднее количество единиц 1 товара в заказе: {:.0f} шт.'.format(df['quantity'].mean()))
print('Медианное значение количества единиц 1 товара в заказе: {:.0f} шт.'.format(df['quantity'].median()))

In [ ]:
df.groupby(['customer_id', 'order_id', 'product']).count().sort_values('date', ascending=False).head()

In [ ]:
print('Количесто заказов до чистки от дубликатов: {} шт.'.format(len(df)))

In [ ]:
df = df.drop_duplicates(subset=['customer_id', 'order_id', 'product']).reset_index(drop=True)

In [ ]:
df.groupby(['customer_id', 'order_id', 'product']).count().sort_values('date', ascending=False).head()

In [ ]:
print('Количесто заказов после чистки от дубликатов: {} шт.'.format(len(df)))

Благодаря группировке по id покупателя, id заказа и товарам в заказе, выяснилось, что в датасете есть дубли. Они могут исказить графики и метрики, поэтому их нужно убрать.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,8))

ax[0].boxplot(df['quantity'])
ax[0].set_title('Распределение количества единиц 1 товара')
ax[0].set_ylabel('Количество, шт.')

ax[1].hist(df['quantity'], bins=60)
ax[1].set_title('Гистограмма распределения количества единиц 1 товара')
ax[1].set_xlabel('Количество, шт.')
ax[1].set_ylabel('Частота')

plt.show()

In [ ]:
cnt_orders = df.groupby('customer_id').agg({'order_id':'count'}).reset_index()
cnt_orders.head()

In [ ]:
df = df.merge(cnt_orders, on='customer_id')

In [ ]:
df.columns = ['date', 'customer_id', 'order_id', 'product', 'quantity', 'price',
       'time', 'month', 'weekday', 'month_number', 'amount_orders']

In [ ]:
df[(df['quantity'] >= 100) & (df['amount_orders'] == 1)]

In [ ]:
print('Количество "оптовиков": {} шт.'.format(len(df[(df['quantity'] >= 100) & (df['amount_orders'] == 1)])))

UPD: Я добавила число заказов, которое совершил покупатель за весь год и отобрала только тех, что совершил 1 покупку и количество единиц 1 товара было 100 и более шт. Их я засчитываю как оптовиков.

На графиках есть выбросы от 100 до 1000 единиц 1 товара, скорее всего это оптовики. Т.к. у нас магазин ориентирован на B2C сегмент и данные показывают, что в обычно берут 1-2 единицы одного товара. Оптовики — это не наша целевая аудитория, поэтому нужно исключить этих пользователей, чтобы они не смазывали нам картину. Тем более что таких покупателей немного(15 шт.)

In [ ]:
anom_users = df[(df['quantity'] >= 100) & (df['amount_orders'] == 1)]['customer_id']
df = df[np.logical_not(df['customer_id'].isin(anom_users))].reset_index()

Проверим сколько уникальных товаров в корзине. Это может помочь понять из какого количества позиций состоит типичный заказ. Посмотрим много ли покупателей берут больше "обычного".

In [ ]:
cnt_goods = df.groupby('order_id').agg({'product': 'nunique'}).reset_index()

In [ ]:
print('Среднее количество уникальных товаров в заказе: {:.0f} шт.'.format(cnt_goods['product'].mean()))
print('Медианное значение количества уникальных товаров в заказе: {:.0f} шт.'.format(cnt_goods['product'].median()))

In [ ]:
fig = plt.subplots(figsize=(10,8))

plt.boxplot(cnt_goods['product'])
plt.title('Распределение количества уникальных товаров в заказе')
plt.ylabel('Количество , шт.')

plt.show()

In [ ]:
print('Количество заказов на больше, чем 10 позиций: {} шт. или {:.2%}'.format(len(cnt_goods[cnt_goods['product'] >= 10]), 
                                                              len(cnt_goods[cnt_goods['product'] >= 10]) / len(cnt_goods)))

Средняя и медиана говорят о том, что обычно покупатели берут 1-2 уникальных товара, но по распределению на ящике с усами мы видим, что есть и те, у кого в заказе может быть и больше 10 позиций. Таких покупателей немного, почти 2% от всех заказов.

In [ ]:
print('Среднее значение цены на товары: {:.2f} руб.'.format(df['price'].mean()))
print('Медианное значение цены на товары: {:.2f} руб.'.format(df['price'].median()))

In [ ]:
print('Квартили:')
df['price'].quantile([0.25, 0.5, 0.75, 1])

Товары по ценам:
    * мелочь - от 1 руб до 150 руб.
    * средние по цене товары от 150 до 500 руб.
    * дорогие от 501 до 2000 руб.
    * самые дорогие от 2001 до 15000 руб.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,8))

ax[0].boxplot(df['price'])
ax[0].set_title('Распределение цен на товары по заказам')
ax[0].set_ylabel('Цены, руб.')

ax[1].boxplot(df['price'])
ax[1].set_title('Распределение цен на товары по заказам до 1000 руб.')
ax[1].set_ylabel('Цены, руб.')
plt.ylim(0,1000)

plt.show()

In [ ]:
loyal_users = df.groupby('customer_id').agg({'order_id':'nunique'}).reset_index()
loyal_users.columns = ['customer_id', 'num_orders']
loyal_users.head()

In [ ]:
print('Количество покупателей, которые сделали 1 заказ: {} шт.'.format(len(loyal_users[loyal_users['num_orders'] == 1])))
print('Количество покупателей, которые сделали больше 1 заказа: {} шт.'.format(len(loyal_users[loyal_users['num_orders'] > 1])))

### Вывод по заказам

За весь год очень много было куплено товаров в ценовой категории до 500 руб. Это как раз около нашего среднего значения 462 руб. Обычно покупатели берут 1-2 позиции по 1-3 шт. 

### Когортный анализ

In [ ]:
first_buy = df.groupby('customer_id')['date'].min()
first_buy.name = 'first_buy'
df = df.merge(first_buy, on='customer_id')

In [ ]:
df['first_month'] = df['first_buy'].astype('datetime64[M]')
df['cohort_lifetime'] = df['month'] - df['first_month']
df['cohort_lifetime'] = (df['cohort_lifetime'] / np.timedelta64(1,'M'))
df['cohort_lifetime'] = df['cohort_lifetime'].round().astype('int64')

In [ ]:
cohort = df.groupby(['first_month','cohort_lifetime']).agg({'customer_id':'nunique'}).reset_index()

In [ ]:
initial_users_count = cohort[cohort['cohort_lifetime'] == 0][['first_month','customer_id']]
initial_users_count = initial_users_count.rename(columns={'customer_id':'cohort_users'})

In [ ]:
cohort = cohort.merge(initial_users_count, on='first_month')

In [ ]:
cohort['retention'] = cohort['customer_id'] / cohort['cohort_users']

In [ ]:
retention_pivot = cohort.pivot_table(index='first_month',columns='cohort_lifetime',values='retention',aggfunc='sum')

In [ ]:
plt.figure(figsize=(15, 10))
plt.title('Возвращаемость посетителей на сервис')
sns.heatmap(retention_pivot, annot=True, fmt='.1%', linewidths=0.5, linecolor='gray', vmax=0.05)
plt.show()

Проанализировав график с когортами, можно придти к выводу, что возвращаемость для нашего сайта это слабое место. 

Только 1 когорта(октябрь 2018 г.) показали самую длительную возвращаемость(lifetime = 5 месяцев). В остальных непродолжительный период либо же дыры, как например, в когорте от мая 2019 г. Пользователи возвращались за покупками в июне, августе и октябре. Самая большая в процетном соотношении возвращаемость была у мартовской когорты 2019 г. на следующий месяц апрель 3,1%. В общем, начиная с января и заканчивая маем, возвращаемость находится на самом большом уровне за весь год. Очевидно, что это влияние сезона.

### Вывод по исследовательскому анализу
У нас низкая возвращаемость в магазин. Покупатели в основном берут по 1-2 позициям 1-3 шт. средний чек заказа 462 руб. Учитывая, что в среднем в месяц около 200 покупок, то магазину есть куда расти. 

## Сегментация покупателей на основе покупок <a id='step3'></a>

Т.к. пока неизвестно в каком виде будет рекомендательная система работать: письма на почту, или уведомления, или на сайте будет подсказка, то для начала нужно понять какие у нас есть категории покупатели.
Я хочу рассмотреть как влияет на распределение цен товаров количество совершенных заказов за период, то, сколько в среднем товаров в корзине, влияет ли когда покупает клиент (в сезон или нет) и влияние того, какие по цене товары покупает клиент(дешевые или нет).

In [ ]:
df['sum_product'] = df['quantity'] * df['price']

In [ ]:
cnt_goods = cnt_goods.merge(df[['customer_id', 'order_id']], on='order_id', how='left')

In [ ]:
mean_goods = cnt_goods.groupby('customer_id').agg({'product':'mean'}).reset_index()

In [ ]:
one_goods_users = mean_goods[mean_goods['product'] <= 2]["customer_id"]
many_goods_users = mean_goods[mean_goods['product'] > 2]["customer_id"]

In [ ]:
one_buy_users = loyal_users[loyal_users["num_orders"] == 1]["customer_id"]
many_buy_users = loyal_users[loyal_users["num_orders"] > 1]["customer_id"]

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,15))

ax[0, 0].boxplot(df.query('customer_id in @one_goods_users')['price'])
ax[0, 0].set_title('Цены на товары в заказах, где было в среднем не больше 2 позиций')
ax[0, 0].set_ylabel('Цены, руб.')

ax[0,1].boxplot(df.query('customer_id in @many_goods_users')['price'])
ax[0,1].set_title('Цены на товары в заказах, где было в среднем больше 2 позиций')
ax[0,1].set_ylabel('Цены, руб.')

ax[1,0].boxplot(df.query('customer_id in @one_goods_users')['price'])
ax[1,0].set_title('Цены на товары в заказах, где было в среднем не больше 2 позиций')
ax[1,0].set_ylabel('Цены, руб.')
ax[1,0].set_ylim([0, 2600])

ax[1,1].boxplot(df.query('customer_id in @many_goods_users')['price'])
ax[1,1].set_title('Цены на товары в заказах, где было в среднем больше 2 позиций')
ax[1,1].set_ylabel('Цены, руб.')
ax[1,1].set_ylim([0, 2500])

plt.show()

Судя по распределениям посетители, которые берут 1-2 позиции покупают более дорогие товары, чем те, у кого в среднем получается больше 2 позиций в заказе. Предлагаю это различие использовать для нашей сегментации.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,15))

ax[0,0].boxplot(df.query('customer_id in @one_buy_users')['price'])
ax[0,0].set_title('Цены на товары в заказах, которые были сделаны 1 раз')
ax[0,0].set_ylabel('Цены, руб.')


ax[0,1].boxplot(df.query('customer_id in @many_buy_users')['price'])
ax[0,1].set_title('Цены на товары в заказах, которые были сделаны > 1 раза')
ax[0,1].set_ylabel('Цены, руб.')


ax[1,0].boxplot(df.query('customer_id in @one_buy_users')['price'])
ax[1,0].set_title('Цены до 2000 руб. на товары в заказах, которые были сделаны 1 раз')
ax[1,0].set_ylabel('Цены, руб.')
ax[1,0].set_ylim([0, 2000])

ax[1,1].boxplot(df.query('customer_id in @many_buy_users')['price'])
ax[1,1].set_title('Цены до 2000 руб. на товары в заказах, которые были сделаны > 1 раза')
ax[1,1].set_ylabel('Цены, руб.')
ax[1,1].set_ylim([0, 2000])

plt.show()

In [ ]:
segment_1 = df.query('customer_id in @one_goods_users')
segment_2 = df.query('customer_id in @many_goods_users')

In [ ]:
print('Количество пользователей, которые в среднем покупают 1-2 товара: {} шт. или {:.0%} от общего количества'\
      .format(segment_1['customer_id'].nunique(), (segment_1['customer_id'].nunique() / df['customer_id'].nunique())))
print('Количество пользователей, которые в среднем покупают много товаров: {} шт. или {:.0%} от общего количества'\
      .format(segment_2['customer_id'].nunique(), (segment_2['customer_id'].nunique() / df['customer_id'].nunique())))
print('Итого: {} шт.'.format(segment_1['customer_id'].nunique() + segment_2['customer_id'].nunique()))
print('----------------------------')
print('Вcего покупателей в первоначальном датафрейме: {} шт.'.format(df['customer_id'].nunique()))

In [ ]:
segment_3 = df.query('customer_id in @one_buy_users')
segment_4 = df.query('customer_id in @many_buy_users')

In [ ]:
print('Количество пользователей, которые покупают 1 раз в год: {} шт. или {:.0%} от общего количества'\
      .format(segment_3['customer_id'].nunique(), (segment_3['customer_id'].nunique() / df['customer_id'].nunique())))
print('Количество пользователей, которые покупают больше 1 раза в год: {} шт. или {:.0%} от общего количества'\
      .format(segment_4['customer_id'].nunique(), (segment_4['customer_id'].nunique() / df['customer_id'].nunique())))
print('Итого: {} шт.'.format(segment_3['customer_id'].nunique() + segment_4['customer_id'].nunique()))
print('----------------------------')
print('Вcего покупателей в первоначальном датафрейме: {} шт.'.format(df['customer_id'].nunique()))

Ниже попробую сегментацию другую:
* новенькие - те, кто совершил покупку единожды за весь год и купил 1-2 товара
* закупы - те, кто совершил покупку единожды за весь год и заказал больше 2 товаров за раз
* бывалые - те, кто покупает у нас чаще 1 раза в год и берет больше 2 товаров в заказе
* лояльные - те, кто покупает у нас чаще 1 раза в год, но берет 1-2 товара

In [ ]:
new_users = df.query('customer_id in @one_buy_users & customer_id in @one_goods_users')
purchase_users = df.query('customer_id in @one_buy_users & customer_id in @many_goods_users')
skilled_users = df.query('customer_id in @many_buy_users & customer_id in @many_goods_users')
loayl_users = df.query('customer_id in @many_buy_users & customer_id in @one_goods_users')

In [ ]:
print('Количество новеньких: {} шт. или {:.0%} от общего количества'\
      .format(new_users['customer_id'].nunique(), (new_users['customer_id'].nunique() / df['customer_id'].nunique())))
print('Количество закупов: {} шт. или {:.0%} от общего количества'\
      .format(purchase_users['customer_id'].nunique(), (purchase_users['customer_id'].nunique() / df['customer_id'].nunique())))
print('Количество бывалых: {} шт. или {:.0%} от общего количества'\
      .format(skilled_users['customer_id'].nunique(), (skilled_users['customer_id'].nunique() / df['customer_id'].nunique())))
print('Количество лояльных: {} шт. или {:.0%} от общего количества'\
      .format(loayl_users['customer_id'].nunique(), (loayl_users['customer_id'].nunique() / df['customer_id'].nunique())))
print('Итого: {} шт.'.format(new_users['customer_id'].nunique() + purchase_users['customer_id'].nunique() + 
                            skilled_users['customer_id'].nunique() + loayl_users['customer_id'].nunique()))
print('----------------------------')
print('Вcего покупателей в первоначальном датафрейме: {} шт.'.format(df['customer_id'].nunique()))

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,15))

ax[0,0].boxplot(new_users['price'])
ax[0,0].set_title('Цены на товары в заказах, которые были сделаны новенькими')
ax[0,0].set_ylabel('Цены, руб.')
ax[0,0].set_ylim([0, 8000])


ax[0,1].boxplot(purchase_users['price'])
ax[0,1].set_title('Цены на товары в заказах, которые были сделаны закупами')
ax[0,1].set_ylabel('Цены, руб.')
ax[0,1].set_ylim([0, 8000])


ax[1,0].boxplot(skilled_users['price'])
ax[1,0].set_title('Цены на товары в заказах, которые были сделаны бывалыми')
ax[1,0].set_ylabel('Цены, руб.')
ax[1,0].set_ylim([0, 8000])


ax[1,1].boxplot(loayl_users['price'])
ax[1,1].set_title('Цены на товары в заказах, которые были сделаны дояльными')
ax[1,1].set_ylabel('Цены, руб.')
ax[1,1].set_ylim([0, 8000])

plt.show()

На графике группы между собой похожи крест-накрест: новенькие похожи на лояльных, а бывалые на закупщиков.

Сегментация на основе сезона:
* сезонники - покупают только в сезон декабь, февраль и март
* остальные - покупают в течение года, т.е. исключаем пользователей, которые берут только в сезон

In [ ]:
season_month = [12, 2, 3]
not_season_month = [1, 4, 5, 6, 7, 8, 9, 10, 11]

In [ ]:
season_users = df.query('month_number in @season_month & month_number not in @not_season_month')
exception_users = season_users['customer_id'].unique()

In [ ]:
other_users = df.query('customer_id not in @exception_users')

In [ ]:
print('Количество сезонных покупателей: {} шт. или {:.0%} от общего количества'\
      .format(season_users['customer_id'].nunique(), (season_users['customer_id'].nunique() / df['customer_id'].nunique())))
print('Количество круглогодичных покупателей: {} шт. или {:.0%} от общего количества'\
      .format(other_users['customer_id'].nunique(), (other_users['customer_id'].nunique() / df['customer_id'].nunique())))
print('Итого: {} шт.'.format(season_users['customer_id'].nunique() + other_users['customer_id'].nunique()))
print('----------------------------')
print('Вcего покупателей в первоначальном датафрейме: {} шт.'.format(df['customer_id'].nunique()))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,8))

ax[0].boxplot(season_users['price'])
ax[0].set_title('Цены на товары в заказах, которые были сделаны в сезон')
ax[0].set_ylabel('Цены, руб.')
ax[0].set_ylim([0, 8000])


ax[1].boxplot(other_users['price'])
ax[1].set_title('Цены на товары в заказах, которые были сделаны круглогодично')
ax[1].set_ylabel('Цены, руб.')
ax[1].set_ylim([0, 8000])

plt.show()

В распределении цен по сезонникам и остальным покупателям получилось, что медианы находятся почти на одном и том же уровне, у сезонников цены на товары в основной своей массе находятся на другом уровне от 50 до 1700 руб, в то время как у остальных от 50 до 1000 руб.

Сегментация на основе цен товаров:
* те, кто покупает мелочь от 1 руб до 150 руб.
* те, кто покупает средние по цене товары от 150 до 500 руб.
* те, кто покупает дорогие товары от 501 до 2000 руб.
* те, кто покупает самые дорогие товары от 2001 до 15000 руб.

Брать буду среднюю цену заказов у покупателя.

In [ ]:
orders_price = df.groupby('order_id').agg({'price':'mean'}).reset_index()

In [ ]:
orders_price = orders_price.merge(df[['order_id', 'customer_id']], on ='order_id', how='left')

In [ ]:
mean_price_users = orders_price.groupby('customer_id').agg({'price':'mean'}).reset_index()

In [ ]:
chep_users = mean_price_users.query('price <= 150')['customer_id']
midle_users = mean_price_users.query('price > 150 & price <= 500')['customer_id']
expensive_users = mean_price_users.query('price > 500 & price <= 2000')['customer_id']
most_expen_users = mean_price_users.query('price > 2000')['customer_id']

In [ ]:
chep = df.query('customer_id in @chep_users')
midle = df.query('customer_id in @midle_users')
expensive = df.query('customer_id in @expensive_users')
most_expen = df.query('customer_id in @most_expen_users')

In [ ]:
print('Количество покупателей дешевых товаров: {} шт. или {:.0%} от общего количества'\
      .format(chep['customer_id'].nunique(), (chep['customer_id'].nunique() / df['customer_id'].nunique())))
print('Количество покупателей средних по цене товаров: {} шт. или {:.0%} от общего количества'\
      .format(midle['customer_id'].nunique(), (midle['customer_id'].nunique() / df['customer_id'].nunique())))
print('Количество покупателей дорогих товаров: {} шт. или {:.0%} от общего количества'\
      .format(expensive['customer_id'].nunique(), (expensive['customer_id'].nunique() / df['customer_id'].nunique())))
print('Количество покупателей самых дорогих товаров: {} шт. или {:.0%} от общего количества'\
      .format(most_expen['customer_id'].nunique(), (most_expen['customer_id'].nunique() / df['customer_id'].nunique())))
print('Итого: {} шт.'.format(chep['customer_id'].nunique() + midle['customer_id'].nunique() + 
                            expensive['customer_id'].nunique() + most_expen['customer_id'].nunique()))
print('----------------------------')
print('Вcего покупателей в первоначальном датафрейме: {} шт.'.format(df['customer_id'].nunique()))

## Вывод по сегментации

Группы получились не однородными по количеству. Это нормально, потому что сегменты отличаются друг от друга по характеру покупок. Но нужно понять есть ли между ними статистическое различие в метрике, которая важна бизнесу.

## Формулировка гипотез и их проверка <a id='step4'></a>

Я сделала несколько сегмантаций, описанных выше. Нужно проверить равны ли эти группы. Т.к. рекомендательная система должна приносить прибыль, то и сравнивать эти группы я буду по средней сумме заказа.

UPD(я в функции ниже также изменила, чтобы учитывать сколько товар стоил в заказе с учетом количества, а не просто цену товара):

**Нулевая гипотеза:** средняя сумма заказа двух групп равны между собой

**Альтернативная гипотеза:** средняя сумма заказа двух групп НЕ равны между собой

In [ ]:
#нужно сделать функцию, чтобы проверить группы между собой вне зависимости от количества групп

alpha = .01 #статистический уровень значимости

def check_sum_order(lst_data):
    for i in range(len(lst_data)):
        for j in lst_data[i+1:]:
            sample1 = lst_data[i].groupby('order_id').agg({'sum_product': 'sum'}).reset_index()['sum_product']
            sample2 = j.groupby('order_id').agg({'sum_product': 'sum'}).reset_index()['sum_product']
            result = st.ttest_ind(sample1, sample2)
            print('------------------------')
            print('p-value:', result.pvalue)
            if (result.pvalue < alpha):
                print("Отвергаем нулевую гипотезу о равенстве, группы между собой различаются")
            else:
                print("Не получилось отвергнуть нулевую гипотезу о равенстве, группы похожи") 

In [ ]:
print('Проверка сегментов разделенных по среднему количеству купленных товаров')
check_sum_order([segment_1, segment_2])

In [ ]:
print('Проверка сегментов разделенных по количеству заказов за год')
check_sum_order([segment_3, segment_4])

In [ ]:
print('Проверка сегментов разделенных по среднему количеству купленных товаров и по количеству заказов за год')
check_sum_order([new_users, purchase_users, skilled_users, loayl_users])

In [ ]:
print('Проверка сегментов разделенных по месяцам покупок')
check_sum_order([season_users, other_users])

In [ ]:
print('Проверка сегментов разделенных по цене товаров')
check_sum_order([chep, midle, expensive, most_expen])

Я проверяла тестом Стьюдента, который проверяет равность средних двух генеральных совокупностей. Из 5 тестов только последний показал различие между группами. Это была сегментация по тому какие в среднем по цене товары берут покупатели. Значение p-value между дорогими и дешевыми составляет 9,4, а между средними по цене и дорогими 9,6, т.е. разница

## Общий вывод к проекту и рекомендации <a id='step5'></a>

В ходе этого проекта я рассматривала заказы, совершенные с 1 октября 2018 по 31 октября 2019 года. Благодаря этому выяснились следующие моменты:
* Есть характерная сезонность это декабрь, февраль и март. И активность в понедельник в 11 часов.
* В основном покупатели берут 1-2 позиции от 1-3 шт. Но также есть и, так называемые, "оптовики", которые закупаются большими партиями.
* У нас низкая возвращаемость на сервис. Отчасти это связанно с сезонностью, но я считаю, что улучшение этого показателя может принести прибыль. 

Я разделила пользователей на группы по средней цене за 1 товар в заказе, т.е. есть покупатели, которые берут дешевые товары, средние по цене, дорогие и самые дорогие товары.

Рекомендательная система может использовать эту информацию следующим образом: для тех кто покупает дорогие и самые дорогие товары предлагать всякую мелочь, сумма их корзины увеличить несильно и психологически им будет комфортно, ведь они купили что-то дорогое и дополнительно взяли безделушку. Для тех кто берет в основном дешевые товары предлагать их больше, потому что эти товары комфортны для этой категории покупателей.

Более качественная рекомендательная система должна будет учитывать предпочтения по категориям товаров.

### Рекомендации

* Улучшение возвращаемости на сервис. Необязательно возвращаемость каждый месяц, мы этого, скорее всего, не добьемся, потому что сезон играет большую роль. Но можно проработать систему напоминаний для ежегодного возвращения. Таким образом, у покупателя будет "под рукой" интернет-магазин, где он каждый год совершает хотя бы одну покупку. Это могут быть ежегодные письма-напоминания о том, что нужно закупиться.
* Если мы узнаем какие категории товаров часто покупает пользователь и тем более если у него есть любимые товары, то можно уведомлять о том, что товар скоро закончится либо же он стал доступен. 